## Importation des bibliothèques

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

## Importation des données

In [2]:
uploaded = files.upload()
import pandas as pd

Saving TSA_Energie.csv to TSA_Energie.csv


In [23]:
df = pd.read_csv('TSA_Energie.csv')
df.head()

,Date_time,Valeur
0,2017-01-01 00:00:00,335.292319
1,2017-01-01 01:00:00,366.188139
2,2017-01-01 02:00:00,344.493160
3,2017-01-01 03:00:00,347.964323
4,2017-01-01 04:00:00,347.877341


## Extraction de la date, du mois, de l'année et semaine

In [24]:
df.dtypes
df['Date'] = pd.to_datetime(df['Date_time'])

In [26]:
df['Day'] = df['Date'].dt.day
df['Mois'] = df['Date'].dt.month
df['Année'] = df['Date'].dt.year
df['Semaine'] = df['Date'].dt.isocalendar().week  # Utilise isocalendar pour éviter les avertissements
df['Jour_semaine'] = df['Date'].dt.dayofweek
df = df.drop(['Date_time', 'Date'], axis=1)
df.head()

,Valeur,Day,Mois,Année,Semaine,Jour_semaine
0,335.292319,1,1,2017,52,6
1,366.188139,1,1,2017,52,6
2,344.493160,1,1,2017,52,6
3,347.964323,1,1,2017,52,6
4,347.877341,1,1,2017,52,6


## Split des données

In [27]:
from sklearn.model_selection import train_test_split

X, y = df.drop('Valeur', axis=1), df['Valeur']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8, random_state=42)

## Implémentation du modèle XGBoost

In [30]:
from re import M
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

model = XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

Mean Squared Error: 450.9037569566226
Root Mean Squared Error: 21.2344945067365


# Aggrégation journalière

## Importation des données

In [89]:
data = pd.read_csv('TSA_Energie.csv')
data.head()

,Date_time,Valeur
0,2017-01-01 00:00:00,335.292319
1,2017-01-01 01:00:00,366.188139
2,2017-01-01 02:00:00,344.493160
3,2017-01-01 03:00:00,347.964323
4,2017-01-01 04:00:00,347.877341


## indexation et aggrégation

In [90]:
data['Date'] = pd.to_datetime(data['Date_time'])

In [91]:
data = data.set_index('Date')

In [92]:
df_daily = data.resample('D').sum().reset_index()

In [93]:
df_daily.head()

,Date,Date_time,Valeur
0,2017-01-01,2017-01-01 00:00:002017-01-01 01:00:002017-01-...,8075.594184
1,2017-01-02,2017-01-02 00:00:002017-01-02 01:00:002017-01-...,8416.983542
2,2017-01-03,2017-01-03 00:00:002017-01-03 01:00:002017-01-...,8827.971231
3,2017-01-04,2017-01-04 00:00:002017-01-04 01:00:002017-01-...,8895.874285
4,2017-01-05,2017-01-05 00:00:002017-01-05 01:00:002017-01-...,8951.426492


In [94]:
df_daily = df_daily.drop('Date_time', axis=1)

In [95]:
df_daily.head()

,Date,Valeur
0,2017-01-01,8075.594184
1,2017-01-02,8416.983542
2,2017-01-03,8827.971231
3,2017-01-04,8895.874285
4,2017-01-05,8951.426492


In [96]:
df_daily['Année'] = df_daily['Date'].dt.year
df_daily['Mois'] = df_daily['Date'].dt.month
df_daily['Semaine'] = df_daily['Date'].dt.isocalendar().week
df_daily['Jour_semaine'] = df_daily['Date'].dt.dayofweek

In [97]:
df_daily.head()

,Date,Valeur,Année,Mois,Semaine,Jour_semaine
0,2017-01-01,8075.594184,2017,1,52,6
1,2017-01-02,8416.983542,2017,1,1,0
2,2017-01-03,8827.971231,2017,1,1,1
3,2017-01-04,8895.874285,2017,1,1,2
4,2017-01-05,8951.426492,2017,1,1,3


## Split des données

In [98]:
from sklearn.model_selection import train_test_split

X_day, y_daily = df_daily.drop(['Date','Valeur'], axis=1), df_daily['Valeur']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_day, y_daily,
                                                    train_size=0.8, random_state=42)

## Implémentation du modèle

In [99]:
from re import M
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

model = XGBRegressor()
model.fit(X_train1, y_train1)
y_pred1 = model.predict(X_test1)
mse = mean_squared_error(y_test1, y_pred1)
rmse = np.sqrt(mse)
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

Mean Squared Error: 64998.30962253397
Root Mean Squared Error: 254.94766055513034
